In [38]:
from arcgis.gis import GIS
import pandas as pd
from dotenv import load_dotenv
import os
from datetime import date
import time

In [163]:
load_dotenv(".env")

True

In [164]:
gis = GIS(
    "https://bwf.maps.arcgis.com/",
    username=os.getenv("UNAME"),
    password=os.getenv("PASSWORD"),
)

In [181]:
ordersFeatureLayer = gis.content.get(os.getenv("INVSURVEY")).layers[0]

def get_raw_orders(as_sdf=True):
    if as_sdf:
        return ordersFeatureLayer.query().sdf
    return ordersFeatureLayer.query().features

get_raw_orders().sample(1).iloc[:,[0,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]
get_raw_orders().loc[orders["objectid"] == 13].iloc[0]

KeyError: 'objectid'

In [183]:
orders = get_raw_orders()
order = orders.loc[orders["objectid"] == 13].iloc[0]
order.Products

'PMLkit1010'

In [179]:
inv_dict = get_raw_inventory().set_index('ShortDesc').loc[:,"Quantity"].to_dict()

inv_dict

{'backpack': 1,
 'incub_bag': 276,
 'whirl_bag': 10,
 'ziploc_bag': 160,
 'aa_battery': 0,
 'health_club_book': 63,
 'kisi_book': 13,
 'calendar_25': 2850,
 'cardboard_set': 70,
 'certificate_comp': 0,
 'flyer': 0,
 'flyer_kisi': 230,
 'flyer_hands': 16,
 'hat': 0,
 'PML1010': 121,
 'bucket_label': 2225,
 'bucket_label_ga': 0,
 'uv_light': 10,
 'manual_ed_24': 30,
 'manual_train_24': 24,
 'marker': 333,
 'pencil': 420,
 'pipette_10': 40,
 'rubber_band': 2500,
 'polo_shirt': 4,
 'tee_shirt': 5,
 'spreader': 18,
 'oasis_50': 10400,
 'petrifilm': 80,
 'colilert': 380,
 'chlorine_test_50': 42,
 'umbrella': 8,
 'veronica_bucket': 500}

In [171]:
inventoryFeatureLayer = gis.content.get(os.getenv("INVDATA")).tables[0]

def get_raw_inventory(as_sdf=True):
    if as_sdf:
        return inventoryFeatureLayer.query().sdf
    return inventoryFeatureLayer.query().features

get_raw_inventory()

,OBJECTID,ShortDesc,LongDesc,Quantity
0,3,backpack,SWE backpack,1
1,4,incub_bag,"Bag, Incubation",276
2,5,whirl_bag,"Bag, Whirlpack",10
3,6,ziploc_bag,"Bag, Ziplock, Gallon",160
4,7,aa_battery,Battery AA,0
5,8,health_club_book,"Book, Health Club Activity Guide",63
6,9,kisi_book,"Book, Kisi and Kofi",13
7,10,calendar_25,"Calendar, 2025",2850
8,11,cardboard_set,"Cardboard, set of 2, with rubber bands",70
9,12,certificate_comp,"Certificate, Completion",0


In [117]:
def add_inventory_item(data):
    correct_format = {'attributes':data}
    result = inventoryFeatureLayer.edit_features(adds=[correct_format])
    return result
    
data =  {'ShortDesc':'veronica_bucket',
        'LongDesc':'Veronica Bucket, Bright Water',
        'Quantity':500,}
add_inventory_item(data)

{'addResults': [{'objectId': 35,
   'uniqueId': 35,
   'globalId': None,
   'success': True}],
 'updateResults': [],
 'deleteResults': []}

In [151]:
def change_inventory_qty(items_to_change):
    raw_inv = get_raw_inventory(False)
    final_updates = []
    for k,v in items_to_change.items():
        item_feature = [f for f in raw_inv if f.attributes['ShortDesc']==k][0]
        item_feature.attributes['Quantity'] = v
        final_updates.append(item_feature)
    result = inventoryFeatureLayer.edit_features(updates=final_updates)
    return result        

In [156]:
change_inventory_qty({'backpack':1,'incub_bag':276})

{'addResults': [],
 'updateResults': [{'objectId': 3,
   'uniqueId': 3,
   'globalId': None,
   'success': True},
  {'objectId': 4, 'uniqueId': 4, 'globalId': None, 'success': True}],
 'deleteResults': []}

In [157]:
get_raw_inventory()

,OBJECTID,ShortDesc,LongDesc,Quantity
0,3,backpack,SWE backpack,1
1,4,incub_bag,"Bag, Incubation",276
2,5,whirl_bag,"Bag, Whirlpack",10
3,6,ziploc_bag,"Bag, Ziplock, Gallon",160
4,7,aa_battery,Battery AA,0
5,8,health_club_book,"Book, Health Club Activity Guide",63
6,9,kisi_book,"Book, Kisi and Kofi",13
7,10,calendar_25,"Calendar, 2025",2850
8,11,cardboard_set,"Cardboard, set of 2, with rubber bands",70
9,12,certificate_comp,"Certificate, Completion",0


In [37]:
invSurvey = (
    invSurveyObj
    .layers[0]
    .query()
    .sdf
    # .features
)
# look_for_13 = [f for f in invSurvey if f.attributes['objectid']==13][0]
# look_for_13.attributes
invSurvey

,objectid,globalid,logo,instructions,BrightWaterID,Namebwe,designation_hidden,ReceivingSWE,Date,Products,...,int_survey_status,CreationDate,Creator,EditDate,Editor,status,order_edited,last_edited,when_completed,SHAPE
0,13,cddbbb42-eb17-46e5-ba94-48321d567a40,<NA>,<NA>,BW822,Abdulai Fatimatu,Player-Coach,BW822 - Abdulai Fatimatu,2024-06-02 12:00:00,PMLkit1010,...,<NA>,2024-06-02 21:48:44.868000,,2024-11-02 02:53:04.776000,Atiwaghana25,Completed,<NA>,NaT,2024-11-01 19:51:14.273999,"{""x"": -0.6999934, ""y"": 6.3455695, ""spatialRefe..."
1,14,1589ef00-890c-4213-b684-6c3fd49bd9d1,<NA>,<NA>,BW822,Abdulai Fatimatu,Player-Coach,BW793 - Amankwah Adu Meshack,2024-06-02 12:00:00,PMLkit1010,...,<NA>,2024-06-02 21:49:25.263000,,2024-06-02 21:49:25.263000,,<NA>,<NA>,NaT,NaT,"{""x"": -0.7097319513559341, ""y"": 6.343543594703..."
2,15,0f2da910-a629-4a4d-a61c-f9016b8e965a,<NA>,<NA>,BW822,Abdulai Fatimatu,Player-Coach,BW794 - Amponsem Benedicta,2024-06-02 12:00:00,PMLkit1010,...,<NA>,2024-06-02 21:50:04.904000,,2024-06-02 21:50:04.904000,,<NA>,<NA>,NaT,NaT,"{""x"": -0.7097282633185387, ""y"": 6.343519538640..."
3,16,89ac1469-54b5-4304-807e-d473e2a1a482,<NA>,<NA>,BW822,Abdulai Fatimatu,Player-Coach,BW798 - Akoto Maxwell,2024-06-02 12:00:00,PMLkit1010,...,<NA>,2024-06-02 21:50:34.615000,,2024-06-02 21:50:34.615000,,<NA>,<NA>,NaT,NaT,"{""x"": -0.7097383216023445, ""y"": 6.343481694348..."
4,17,096c1ca3-5401-40c4-8d9c-e482c9e88548,<NA>,<NA>,BW822,Abdulai Fatimatu,Player-Coach,BW831 - Otema Bridgette,2024-06-02 12:00:00,PMLkit1010,...,<NA>,2024-06-02 21:51:06.260000,,2024-06-02 21:51:06.260000,,<NA>,<NA>,NaT,NaT,"{""x"": -0.7097326219081879, ""y"": 6.343452441506..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,72,8655ea68-4d1f-4b90-8ae7-396c40111c52,<NA>,<NA>,BW670,Chertor Lucy,Player-Coach,BW698 - Michael Tetteh,2024-09-24 12:00:00,Flyers,...,<NA>,2024-09-27 07:38:16.897000,,2024-09-27 07:38:16.897000,,<NA>,<NA>,NaT,NaT,"{""x"": 0, ""y"": 0, ""spatialReference"": {""wkid"": ..."
60,73,09690c45-44a1-48da-b2e8-4c297572224c,<NA>,<NA>,BW670,Chertor Lucy,Player-Coach,BW698 - Michael Tetteh,2024-10-21 12:00:00,Flyers,...,<NA>,2024-10-23 10:11:15.287000,,2024-10-23 10:11:15.287000,,<NA>,<NA>,NaT,NaT,"{""x"": 0, ""y"": 0, ""spatialReference"": {""wkid"": ..."
61,74,9f2eb8e3-6417-44b2-a3df-b8345eb85794,<NA>,<NA>,BW670,Chertor Lucy,Player-Coach,BW698 - Michael Tetteh,2024-10-23 12:00:00,Flyers,...,<NA>,2024-10-25 06:51:09.342999,,2024-10-25 06:51:09.342999,,<NA>,<NA>,NaT,NaT,"{""x"": 0, ""y"": 0, ""spatialReference"": {""wkid"": ..."
62,75,09b53499-b28f-43bd-aba1-8e77b8d6ca81,<NA>,<NA>,BW670,Chertor Lucy,Player-Coach,BW698 - Michael Tetteh,2024-10-26 12:00:00,Flyers,...,<NA>,2024-10-26 19:14:23.860999,,2024-10-26 19:14:23.860999,,<NA>,<NA>,NaT,NaT,"{""x"": 0, ""y"": 0, ""spatialReference"": {""wkid"": ..."


In [ ]:
for fea in get_raw_orders(False):
    fea.attributes['status'] = "Open"

In [33]:
edited13 = look_for_13
edited13.attributes['status'] = "Completed"
edited13.attributes['when_completed'] = (pd.to_datetime('now') - pd.Timestamp("1970-01-01")) // pd.Timedelta('1ms')
edited13.attributes

{'objectid': 13,
 'globalid': 'cddbbb42-eb17-46e5-ba94-48321d567a40',
 'logo': None,
 'instructions': None,
 'BrightWaterID': 'BW822',
 'Namebwe': 'Abdulai  Fatimatu',
 'designation_hidden': 'Player-Coach',
 'ReceivingSWE': 'BW822 - Abdulai  Fatimatu',
 'Date': 1717329600000,
 'Products': 'PMLkit1010',
 'NoBackpacks': None,
 'NoIncubationBags': None,
 'NoWhirlpaks': None,
 'NoAABatteries': None,
 'NoHealthClub': None,
 'NoKisiKofiBooks': None,
 'NoCalendars': None,
 'NoCardboards': None,
 'NoCertificatesCompletion': None,
 'NoFlyers': None,
 'NoKisiKofiFlyers': None,
 'NoHats': None,
 'NoPML1010': 3,
 'NoPML1510': None,
 'NoPML2525': None,
 'NoBWVeronicaBucketLabels': None,
 'NoGAVeronicaBucketLabels': None,
 'NoUVLights': None,
 'NoEdManuals': None,
 'NoTrainManuals': None,
 'NoMarkers': None,
 'NoPencils': None,
 'NoPipettes': None,
 'NoPosters': None,
 'NoWashHandsPoster': None,
 'NoPoloShirts': None,
 'NoTeeShirts': None,
 'NoWaterSpreaders': None,
 'NoOasisTablets': None,
 'NoColi

In [34]:
feature_layer = invSurveyObj.layers[0]
result = feature_layer.edit_features(updates=[edited13])
result

{'addResults': [],
 'updateResults': [{'objectId': 13,
   'uniqueId': 13,
   'globalId': 'cddbbb42-eb17-46e5-ba94-48321d567a40',
   'success': True}],
 'deleteResults': []}

In [11]:
rename_cols= {'NoPML1010':'PMLkit1010',
                     'NoPetrifilm':'PetrifilmTests',
                     'NoOasisTablets':'PurificationTabletsOasis',
                     'NoHealthClub':'HealthClubExerciseBook',
                     'NoChlorineStrips50':'ChlorineStrips50',
                     'NoFlyers':'Flyers',
                     'NoKisiKofiBooks':'KisiKofiBook',
                     'NoCalendars':'Calendars',
                     'NoWashHandsPoster':'WashHandsPoster',
                     }

In [17]:
features=[{}]
feature_layer = invSurveyObj.layers[0]

In [ ]:
response = feature_layer.edit_features(updates=features)

In [16]:
orders = (
    # pd.DataFrame.spatial.from_featureset(invSurvey)
    pd.DataFrame([s.as_dict["attributes"] for s in invSurvey])
    .rename(columns={"objectid": "order_id"})
    .assign(
        status=lambda df_:df_.status.fillna("Open"),
        Date=lambda df_: pd.to_datetime(df_.Date, unit="ms").dt.strftime("%d %B, %Y"),
    )
    .rename(columns=rename_cols).set_index('order_id')
)
orders

,globalid,logo,instructions,BrightWaterID,Namebwe,designation_hidden,ReceivingSWE,Date,Products,NoBackpacks,...,int_survey_mode,int_survey_status,CreationDate,Creator,EditDate,Editor,status,order_edited,last_edited,when_completed
order_id,,,,,,,,,,,,,,,,,,,,,
13,cddbbb42-eb17-46e5-ba94-48321d567a40,None,None,BW822,Abdulai Fatimatu,Player-Coach,BW822 - Abdulai Fatimatu,"02 June, 2024",PMLkit1010,None,...,new,None,1717364924868,,1717364924868,,Open,None,None,None
14,1589ef00-890c-4213-b684-6c3fd49bd9d1,None,None,BW822,Abdulai Fatimatu,Player-Coach,BW793 - Amankwah Adu Meshack,"02 June, 2024",PMLkit1010,None,...,new,None,1717364965263,,1717364965263,,Open,None,None,None
15,0f2da910-a629-4a4d-a61c-f9016b8e965a,None,None,BW822,Abdulai Fatimatu,Player-Coach,BW794 - Amponsem Benedicta,"02 June, 2024",PMLkit1010,None,...,new,None,1717365004904,,1717365004904,,Open,None,None,None
16,89ac1469-54b5-4304-807e-d473e2a1a482,None,None,BW822,Abdulai Fatimatu,Player-Coach,BW798 - Akoto Maxwell,"02 June, 2024",PMLkit1010,None,...,new,None,1717365034615,,1717365034615,,Open,None,None,None
17,096c1ca3-5401-40c4-8d9c-e482c9e88548,None,None,BW822,Abdulai Fatimatu,Player-Coach,BW831 - Otema Bridgette,"02 June, 2024",PMLkit1010,None,...,new,None,1717365066260,,1717365066260,,Open,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,8655ea68-4d1f-4b90-8ae7-396c40111c52,None,None,BW670,Chertor Lucy,Player-Coach,BW698 - Michael Tetteh,"24 September, 2024",Flyers,None,...,new,None,1727422696897,,1727422696897,,Open,None,None,None
73,09690c45-44a1-48da-b2e8-4c297572224c,None,None,BW670,Chertor Lucy,Player-Coach,BW698 - Michael Tetteh,"21 October, 2024",Flyers,None,...,new,None,1729678275287,,1729678275287,,Open,None,None,None
74,9f2eb8e3-6417-44b2-a3df-b8345eb85794,None,None,BW670,Chertor Lucy,Player-Coach,BW698 - Michael Tetteh,"23 October, 2024",Flyers,None,...,new,None,1729839069343,,1729839069343,,Open,None,None,None
